### Identificação de Forças e Fraquezas em aplicativos de Streamming

>Amazon Prime vs Netflix

>Vamos identificar os principais tópicos (temas, assuntos) presentes em:

>>resenhas positivas - de quem gostou! =D

>> resenhas negativas - de quem não curtiu :(

 - [Código do treinamento usado no TCC](https://colab.research.google.com/drive/1PoTd1W5rkf1YX51CyjFy21KxOfpnxq61#scrollTo=4uQTbG3UZp2-)

 - [Código do teste e extração dos tópicos usado no TCC](https://colab.research.google.com/drive/1FWPUqCPoa0z2X8hi38d2R59yHJnBaynk#scrollTo=193nBjvX5RIk)
 

## Importando bibliotecas




In [ ]:
pip install --upgrade numpy

In [ ]:
pip install --upgrade pandas

In [ ]:
pip install --upgrade pyLDAvis

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('gutenberg')
nltk.download('punkt')

from unicodedata import normalize
from nltk.probability import FreqDist

import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
import time
from gensim.models import CoherenceModel
#plot
import pyLDAvis
#import pyLDAvis.gensim
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS

## Montando Repositorio do Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/DataScience/AnaliseSentimento-App/BaseRotulada/

/content/drive/My Drive/DataScience/AnaliseSentimento-App/BaseRotulada


In [4]:
ls

df_amazon_rotulada_modelo.csv  df_netflix_rotulada_modelo.csv


In [5]:
caminho_stopwords = "/content/drive/My Drive/GRADUAÇÃO UFABC/PGC/Desenvolvimento/AnaliseExploratoria/stopwords.txt"

## Codigo de producao

### ArquivoUtils

In [6]:
# -*- coding: utf-8 -*-

import os
import io
import pandas as pd
import requests
from pygments.lexers.csound import newline

def to_string(alist):
  seperator = ' '
  raw_text = seperator.join(alist)
  return raw_text

def filtra_arquivos_ano_edicao(ano_edicao, lista_arquivos):
  lista_filtro = []
  ano_edicao = str(ano_edicao)
  for arq in lista_arquivos:
      if ano_edicao in arq:
          lista_filtro.append(arq)
  return lista_filtro
  
def obter_lista_novas_stopwords(caminho_stopwords):
  novas_stopwords = []
  # print("\n*************************************************************************\n")
  # print("Extraindo StopWords ...\n")
  try:
      arquivo_stopwords = open(caminho_stopwords, "r", encoding="utf-8")
      for linha in arquivo_stopwords:
          linha = linha.strip()
          novas_stopwords.append(linha)
  except:
      print("Erro na extração de StopWords! \n")
  finally:
      arquivo_stopwords.close()
      # print("Extração das stopWords realizada com sucesso! \n")
      # print("*************************************************************************\n")
  return novas_stopwords

def extracao_artigos_listas(nome_do_arquivo):
  caminho_fixo = caminho_corpus
  nome_arq_completo = caminho_fixo + nome_do_arquivo
  arquivo = open(nome_arq_completo + ".txt", "r", encoding="utf-8")
  lista_artigo = []
  for linhas in arquivo:
      lista_artigo.append(linhas)
  arquivo.close()
  return lista_artigo


### PreProcessamento

In [7]:
# -*- coding: utf-8 -*-
import re
from unicodedata import normalize

import nltk
from gensim import corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('gutenberg')
nltk.download('punkt')


def tokenizar_texto(texto):
  texto_tokenizado = word_tokenize(texto.lower())
  return texto_tokenizado

def remover_acentos(texto):
  return normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

def converter_texto_para_minusculo(texto):
  return texto.lower()

def remover_caracteres_especiais(texto):
  texto_sem_carac_esp = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕàÀìÌòÒïÏñÑÜüçÇ ]', "", texto)
  return texto_sem_carac_esp

def remover_email(texto):
  padrao_email = r'[\w+\./{1}]+@(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}'
  texto_sem_email = re.sub(padrao_email, "", texto)
  return texto_sem_email

def remover_url(texto):
  padrao_url = r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*'
  texto_sem_url = re.sub(padrao_url, "", texto)
  return texto_sem_url

def remover_numeros(texto):
  texto_sem_numeros = re.sub(r'\d', "", texto)
  return texto_sem_numeros

def remover_sequencia_espaco_branco(texto):
  padrao = re.compile(r"(\s)\1{1,}")
  return padrao.sub(r"\1", texto)

def remover_palavras_por_tamanho(texto, limite):
  lista_texto = tokenizar_texto(texto)
  lista_palavras_retorno = []
  separador = ' '
  for palavra in lista_texto:
      if len(palavra) > limite:
          lista_palavras_retorno.append(palavra)
  texto_pos_remocao = separador.join(lista_palavras_retorno)
  return texto_pos_remocao


def pre_processamento_2019(texto):
  texto = remover_sequencia_espaco_branco(texto)
  texto = remover_textos_layout(texto)
  texto = remover_enderecos_emails(texto)
  texto = remover_url(texto)
  texto = remover_numeros(texto)
  texto = remover_pronomes_obliquos(texto)
  texto = remover_estados(texto)
  texto = remover_caracteres_especiais(texto)
  texto = remover_sequencia_espaco_branco(texto)
  texto = converter_termos_geral(texto)
  texto = converter_termos_2019(texto)
  texto = remover_stopwords(texto)
  texto = remover_palavras_por_tamanho(texto, 1)
  texto = remover_acentos(texto)
  return texto



def unificar_artigos_com_n_gramas(lista_de_lista_de_artigos):
  artigo_tokens_unificados = []
  for artigo in lista_de_lista_de_artigos:
      xtexto = artigo
      unigramas = xtexto
      xbigramas = construir_bigramas(xtexto)
      xtrigramas = construir_trigramas(xtexto)
      xunigramas = unigramas + xbigramas + xtrigramas
      artigo_tokens_unificados.append(xunigramas)
  return artigo_tokens_unificados

def construir_bigramas(artigo_tokenizado):
  bigrams = []
  for i in range(0, len(artigo_tokenizado)):
      if (i == len(artigo_tokenizado) - 1):
          break
      else:
          bigrama_obs = artigo_tokenizado[i] + '_' + artigo_tokenizado[i + 1]
          bigrams.append(bigrama_obs)
  # colocar analise por frequencia aqui
  bigrams = remover_ngram_por_frequencia(bigrams, 7)
  return bigrams

def construir_trigramas(artigo_tokenizado):
  trigrams = []
  for i in range(0, len(artigo_tokenizado)):
      if (i == len(artigo_tokenizado) - 2):
          break
      else:
          trigrama_obs = artigo_tokenizado[i] + '_' + artigo_tokenizado[i + 1] + '_' + artigo_tokenizado[i + 2]
          trigrams.append(trigrama_obs)
  # colocar analise por frequencia aqui
  trigrams = remover_ngram_por_frequencia(trigrams, 7)
  return trigrams

def remover_ngram_por_frequencia(lista_ngrams, frequencia_limiar):
  fdist = FreqDist(lista_ngrams)
  lista_ngrams_retorno = [w for w in lista_ngrams if fdist[w] >= frequencia_limiar]
  return lista_ngrams_retorno

def obter_dicionario(lista_de_lista_artigos):
  dic_id_palavras = corpora.Dictionary(lista_de_lista_artigos)
  return dic_id_palavras

def obter_corpus(dicionario_artigos, lista_de_lista_artigos):
  corpus_artigo = [dicionario_artigos.doc2bow(lista_artigo) for lista_artigo in lista_de_lista_artigos]
  return corpus_artigo

def obter_lista_geral_todos_artigos():
  lista_artigos_2011 = pre_processamento_lista_artigos(2011)
  lista_artigos_2013 = pre_processamento_lista_artigos(2013)
  lista_artigos_2014 = pre_processamento_lista_artigos(2014)
  lista_artigos_2015 = pre_processamento_lista_artigos(2015)
  lista_artigos_2016 = pre_processamento_lista_artigos(2016)
  lista_artigos_2017 = pre_processamento_lista_artigos(2017)
  lista_artigos_2018 = pre_processamento_lista_artigos(2018)
  lista_artigos_2019 = pre_processamento_lista_artigos(2019)
  lista_completa = lista_artigos_2011 + lista_artigos_2013 + lista_artigos_2014 + lista_artigos_2015 + lista_artigos_2016 + lista_artigos_2017 + lista_artigos_2018 + lista_artigos_2019
  return lista_completa




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Lendo os dataframes das bases consolidadas - seguir daqui

In [11]:
df_netflix = pd.read_csv('df_netflix_rotulada_modelo.csv',
                                     encoding='utf-8')
df_netflix['Review'][3]

'Vou dar três estrelas, por ter descontado assim que cadastrei o cartão, estava no anúncio que era um mês grátis. Mas o app em si é bom, tem vários filmes ótimos.'

In [10]:
df_amazon = pd.read_csv('df_amazon_rotulada_modelo.csv',
                                     encoding='utf-8')
df_amazon['Review'][2]

'É ruim o fato da minha lista ficar junto com os meus assistidos. Legenda demora demais pra passar depois que você da o play.'

# Adequação dos dados do Df

---

Idéia é adequar os dados para a estrutura de Lista contendo uma Lista das resenhas como Strings

Fazer um Laço simples pelo df e dar append em cada item, compondo uma nova lista de str

# Treinamento

In [12]:
def obter_modelo_lda(corpus_resenhas, dicionario_resenhas, numero_de_topicos, hiper_param_alfa):
  inicio = time.time()
  lda_model = gensim.models.LdaMulticore(corpus=corpus_resenhas,
                                          id2word=dicionario_resenhas,
                                          num_topics=numero_de_topicos,
                                          random_state=100,
                                          # update_every=1,
                                          chunksize=100,
                                          passes=50,
                                          alpha=hiper_param_alfa,
                                          eta=0.1,
                                          per_word_topics=True)
  fim = time.time()
  tempo_execucao = round((fim - inicio) / 60)
  if tempo_execucao < 1:
      tempo_execucao = 1
  #print("Execução estimada da obtenção de um objeto de modelo LDA: Até {0} Min(s)".format(tempo_execucao))
  return lda_model

def calcular_coerencia_modelo(modelo_lda, lista_de_lista_resenhas, dicionario):
  #print("Iniciando o Calculo da coerencia de um objeto de modelo LDA")

  modelo_coerencia_lda = CoherenceModel(model=modelo_lda, texts=lista_de_lista_resenhas,
                                        dictionary=dicionario, coherence='c_v')
  valor_coerencia_lda = modelo_coerencia_lda.get_coherence()
  #print("Coerencia calculada com sucesso!\n")
  print("\nCoerencia C_v: {0}".format(valor_coerencia_lda))

  return round(valor_coerencia_lda, 4)


# Extracao do modelo para testes de Analise - Corpus Reduzido (ed. 2014)

In [ ]:
from pprint import pprint
from gensim.test.utils import datapath

import gensim
import time

lista_unificada_2014 = pre_processamento_lista_artigos(2014)

#para testes completo com corpus completo
#lista_lista_teste = lista_funcao
print("Lista de Lista para Analise : {0}".format(len(lista_unificada_2014)))


lista_unificada_ngrams_2014 = unificar_artigos_com_n_gramas(lista_unificada_2014)
print("Lista de Lista Ngrams Para Testes : {0}".format(len(lista_unificada_ngrams_2014)))

lista_de_lista_artigos_unificados_2014 = lista_unificada_ngrams_2014

print("Lista de Lista Artigos da Edicao : {0}".format(len(lista_de_lista_artigos_unificados_2014)))


dicionario_edicao_2014 = obter_dicionario(lista_de_lista_artigos_unificados_2014)
corpus_artigos_2014 = obter_corpus(dicionario_edicao_2014, lista_de_lista_artigos_unificados_2014)
print(len(corpus_artigos_2014[11]))

print("\n\nIniciando o teste do modelo - extração dos tópícos...\n")
#gerar_planilha_treinamento_modelo(corpus_artigos_2013, dicionario_edicao_2013, lista_de_lista_artigos_unificados_2013, "tabela_testes_treino_2013")
meu_modelo_2014 = obter_modelo_lda(corpus_artigos_2014, dicionario_edicao_2014, 7, 0.61)
coer_2014 = calcular_coerencia_modelo(meu_modelo_2014, lista_de_lista_artigos_unificados_2014, dicionario_edicao_2014)
print(coer_2014)
pprint(meu_modelo_2014.print_topics())

print("\n\nTeste Realizado")





# Montando Repositorio do Drive para Analise


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/GRADUAÇÃO UFABC/PGC/Desenvolvimento/AnaliseExploratoria/

/content/drive/My Drive/GRADUAÇÃO UFABC/PGC/Desenvolvimento/AnaliseExploratoria


In [ ]:
ls

AnaliseExploratoria.ipynb  model                  model.id2word  stopwords.txt
lda.html                   model.expElogbeta.npy  model.state


# Graficos LDAvis - Analise


---
Analise eploratoria dos topicos das edições citadas no artigo

Podemos 


In [ ]:

# Visualize the topics
pyLDAvis.enable_notebook(local=True)
vis = pyLDAvis.gensim.prepare(meu_modelo_2014, corpus_artigos_2014, dicionario_edicao_2014, sort_topics=False)
vis

In [ ]:
ls

AnaliseExploratoria.ipynb  model                  model.id2word  stopwords.txt
lda.html                   model.expElogbeta.npy  model.state


# Analise dos Principaios termos ao longo das edições

In [ ]:
def obter_termos_frequencia(lista_de_lista_de_artigos):
  texto2 = []
  for artigo in lista_de_lista_de_artigos:
    xtexto = artigo
    unigramas = xtexto
    xbigramas = construir_bigramas(xtexto)
    xtrigramas = construir_trigramas(xtexto)
    
    xunigramas = unigramas + xbigramas + xtrigramas 
    texto2 = texto2 + xunigramas
  xfrequencia = texto2
  return xfrequencia

Obtendo os 10 termos mais frequentes de todo o corpus

In [ ]:
lista_lista_pre_processada_todos_artigos = obter_lista_geral_todos_artigos()
xfrequencia = obter_termos_frequencia(lista_lista_pre_processada_todos_artigos)
num_freq = 10
fdist_geral = FreqDist(xfrequencia)
print("\n 10 termos mais frequentes em todo o corpus")
print("\n  PALAVRA -- FREQUÊNCIA")
fdist_geral.most_common(num_freq)


 10 termos mais frequentes em todo o corpus

  PALAVRA -- FREQUÊNCIA


[('dados', 2827),
 ('sistema', 2414),
 ('desenvolvimento', 1742),
 ('forma', 1627),
 ('informacao', 1563),
 ('trabalho', 1460),
 ('tecnologia', 1398),
 ('informacoes', 1157),
 ('processo', 1144),
 ('uso', 1119)]

In [ ]:
lista_dez_termos_geral = fdist_geral.most_common(15)

In [ ]:
def obter_termos_freqDist(lista_tuplas_most_common):
  lista_termos = []
  for tupla_freq in lista_tuplas_most_common:
    lista_termos.append(tupla_freq[0])
  return lista_termos



---


Lista dos 10 Termos mais frequentes que colocaremos no gráfico:

---



In [ ]:
lista_dez_termos_geral = obter_termos_freqDist(lista_dez_termos_geral)
print(lista_dez_termos_geral)

['dados', 'sistema', 'desenvolvimento', 'forma', 'informacao', 'trabalho', 'tecnologia', 'informacoes', 'processo', 'uso', 'aplicacao', 'software', 'web', 'sistemas', 'resultados']


In [ ]:
['dados', 'sistema', 'desenvolvimento', 'informacao', 'trabalho', 'tecnologia', 'processo', 'aplicacao', 'software', 'web']


# Obtendo um modelo de distribuição de frequência para cada edição

In [ ]:
def obter_modelo_freq(ano_edicao):
  lista_lista_pre_processada = []
  ano_edicao = str(ano_edicao)
  if ano_edicao == "2011":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2011)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist
  
  elif ano_edicao == "2013":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2013)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

  elif ano_edicao == "2014":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2014)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

  elif ano_edicao == "2015":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2015)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

  elif ano_edicao == "2016":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2016)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

  elif ano_edicao == "2017":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2017)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

  elif ano_edicao == "2018":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2018)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

  elif ano_edicao == "2019":
    lista_lista_pre_processada = pre_processamento_lista_artigos(2019)
    texto_com_todos_termos = obter_termos_frequencia(lista_lista_pre_processada)
    model_fdist = FreqDist(texto_com_todos_termos)
    return model_fdist

In [ ]:
freqDist_2011 = obter_modelo_freq(2011)
freqDist_2013 = obter_modelo_freq(2013)
freqDist_2014 = obter_modelo_freq(2014)
freqDist_2015 = obter_modelo_freq(2015)
freqDist_2016 = obter_modelo_freq(2016)
freqDist_2017 = obter_modelo_freq(2017)
freqDist_2018 = obter_modelo_freq(2018)
freqDist_2019 = obter_modelo_freq(2019)


In [ ]:
freqDist_2011.most_common(20)

def obter_frequencia_termos_cada_edicao(termo):
  lista_ano_freq = []
  lista_ano_freq.append(termo)
  edicoes = [2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
  for ano_ed in edicoes:
    freq_dist_ed = obter_modelo_freq(ano_ed)
    freq_termo = freq_dist_ed[termo]
    lista_ano_freq.append(freq_termo)
  return lista_ano_freq


In [ ]:
obter_frequencia_termos_cada_edicao('dados')

['dados', 88, 499, 626, 460, 361, 366, 167, 260]

In [ ]:
lista_dez_termos = ['dados', 'sistema', 'desenvolvimento', 'informacao', 'trabalho', 'tecnologia', 'processo', 'aplicacao', 'software', 'web']
for termo in lista_dez_termos:
  freq_termos = obter_frequencia_termos_cada_edicao(termo)
  print(freq_termos)

['dados', 88, 499, 626, 460, 361, 366, 167, 260]
['sistema', 84, 374, 386, 214, 431, 388, 309, 228]
['desenvolvimento', 89, 245, 369, 201, 260, 243, 160, 175]
['informacao', 26, 475, 289, 117, 428, 101, 71, 56]
['trabalho', 53, 286, 312, 193, 197, 158, 140, 121]
['tecnologia', 37, 399, 180, 103, 404, 83, 101, 91]
['processo', 19, 200, 242, 183, 211, 146, 81, 62]
['aplicacao', 71, 145, 259, 136, 128, 171, 104, 96]
['software', 90, 142, 276, 169, 135, 69, 65, 87]
['web', 29, 229, 259, 86, 92, 176, 105, 57]


Listas com as frequencias dos termos por edição:


```
['dados', 88, 499, 626, 460, 361, 366, 167, 260]
['sistema', 84, 374, 386, 214, 431, 388, 309, 228]
['desenvolvimento', 89, 245, 369, 201, 260, 243, 160, 175]
['forma', 48, 295, 283, 252, 234, 189, 168, 158]
['informacao', 26, 475, 289, 117, 428, 101, 71, 56]
['trabalho', 53, 286, 312, 193, 197, 158, 140, 121]
['tecnologia', 37, 399, 180, 103, 404, 83, 101, 91]
['informacoes', 48, 213, 196, 207, 187, 130, 69, 107]
['processo', 19, 200, 242, 183, 211, 146, 81, 62]
['uso', 48, 194, 205, 165, 152, 119, 135, 101]

```









Armazenando os dados em um dicionario <chave, valor> 

chave: termo
valor: lista de frequencia por edição

In [ ]:
dict_freq_termos = {
   "dados" : obter_frequencia_termos_cada_edicao('dados')[1:],
   "sistema" : obter_frequencia_termos_cada_edicao('sistema')[1:],
   "desenvolvimento" : obter_frequencia_termos_cada_edicao('desenvolvimento')[1:],
   "informacao" : obter_frequencia_termos_cada_edicao('informacao')[1:],
   "trabalho" : obter_frequencia_termos_cada_edicao('trabalho')[1:],
   "tecnologia" : obter_frequencia_termos_cada_edicao('tecnologia')[1:],
   "processo" : obter_frequencia_termos_cada_edicao('processo')[1:],
   "aplicacao" : obter_frequencia_termos_cada_edicao('aplicacao')[1:],
   "software" : obter_frequencia_termos_cada_edicao('software')[1:],
   "web" : obter_frequencia_termos_cada_edicao('web')[1:],
}

In [ ]:
dict_freq_termos["dados"]

[88, 499, 626, 460, 361, 366, 167, 260]

In [ ]:
dict_freq_termos["web"]

[29, 229, 259, 86, 92, 176, 105, 57]

In [ ]:
dict_freq_termos

{'aplicacao': [71, 145, 259, 136, 128, 171, 104, 96],
 'dados': [88, 499, 626, 460, 361, 366, 167, 260],
 'desenvolvimento': [89, 245, 369, 201, 260, 243, 160, 175],
 'informacao': [26, 475, 289, 117, 428, 101, 71, 56],
 'processo': [19, 200, 242, 183, 211, 146, 81, 62],
 'sistema': [84, 374, 386, 214, 431, 388, 309, 228],
 'software': [90, 142, 276, 169, 135, 69, 65, 87],
 'tecnologia': [37, 399, 180, 103, 404, 83, 101, 91],
 'trabalho': [53, 286, 312, 193, 197, 158, 140, 121],
 'web': [29, 229, 259, 86, 92, 176, 105, 57]}

# Construindo o gráfico da frequencia dos termos por ano de edição

Importando as bibliotecas

In [ ]:
import plotly 
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
cd /content/drive/My Drive/GRADUAÇÃO UFABC/PGC/Desenvolvimento/AnaliseExploratoria/

/content/drive/My Drive/GRADUAÇÃO UFABC/PGC/Desenvolvimento/AnaliseExploratoria


In [ ]:
ls

AnaliseExploratoria.ipynb  lda.html  stopwords.txt


In [ ]:
anos_edicoes = [2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
trace1 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['dados'],
                    mode = 'markers+lines',
                    name = 'dados')

trace2 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['sistema'],
                    mode = 'markers+lines',
                    name = 'sistema')

trace3 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['desenvolvimento'],
                    mode = 'markers+lines',
                    name = 'desenvolvimento')

trace4 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['forma'],
                    mode = 'markers+lines',
                    name = 'forma')


trace5 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['informacao'],
                    mode = 'markers+lines',
                    name = 'informacao')

trace6 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['trabalho'],
                    mode = 'markers+lines',
                    name = 'trabalho')

trace7 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['tecnologia'],
                    mode = 'markers+lines',
                    name = 'tecnologia')

trace8 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['processo'],
                    mode = 'markers+lines',
                    name = 'processo')

trace9 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['aplicacao'],
                    mode = 'markers+lines',
                    name = 'aplicacao')

trace10 = go.Scatter(x = anos_edicoes,
                    y = dict_freq_termos['software'],
                    mode = 'markers+lines',
                    name = 'software')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10]

layout = go.Layout(
    title=go.layout.Title(
        text='Frequência dos termos mais relavantes do EATI (2011-2019)',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Anos',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Frequência',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styling-names')

In [ ]:
fig = plotly.graph_objs.Figure(data=data, layout=layout)



In [ ]:
fig